In [37]:
import sys, os
sys.path.append(os.pardir) # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from deep_learning_github.dataset.mnist import load_mnist
import numpy as np
import pickle

In [38]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

print(x_train.shape) # (60000, 784) : (훈련 이미지, 훈련 레이블)
print(t_train.shape)
print(x_test.shape) # (10000, 784) : (시험 이미지, 시험 레이블)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [39]:
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.int16(img)) # numpy로 저장된 이미지 데이터를 PIL용 데이터 객체로 변환
    pil_img.show()
    
img = x_train[0]
label = t_train[0]
print(label)

5


In [40]:
print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

(784,)
(28, 28)


In [41]:
img_show(img)

In [42]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [58]:
def softmax(x):
    c = np.max(x)
    exp_x = np.exp(x - c)
#     exp_x = np.exp(x)
    exp_sum_x = np.sum(np.exp(x))
    return exp_x / exp_sum_x

In [59]:
def certain_num():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=False, flatten=False, one_hot_label=False)
    return x_test, t_test

In [60]:
np.set_printoptions(threshold=784,linewidth=np.inf)

In [61]:
x, t = certain_num()

print(x[3])
print(t[3])

[[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0  11 150 253 202  31   0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0  37 251 251 253 107   0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0  21 197 251 251 253 107   0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0 110 190 251 251 251 253 169 109  62   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0 253 251 251 251 251 253 251 2

In [62]:
# normalize를 해서 softmax overflow 방지
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

In [63]:
# 학습된 가중치 매개변수 읽기
# 가중치와 편향 매개변수가 딕셔너리 변수로 저장되어 있다.
def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
        
        return network

In [64]:
network = init_network()
print(network)

{'b2': array([-0.01471108, -0.07215131, -0.00155692,  0.12199665,  0.11603302, -0.00754946,  0.04085451, -0.08496164,  0.02898045,  0.0199724 ,  0.19770803,  0.04365116, -0.06518728, -0.05226324,  0.0113163 ,  0.03049979,  0.04060355,  0.0695399 , -0.07778469,  0.0692313 , -0.09365533,  0.0548001 , -0.03843745,  0.02123107,  0.03793406, -0.02806267, -0.01818407,  0.06870425,  0.0542943 ,  0.0674368 ,  0.06264312, -0.0233236 , -0.01589135,  0.01860516,  0.01839287, -0.01568104, -0.07422207, -0.01606729, -0.02262172, -0.01007509,  0.0434415 , -0.12020151,  0.02802471, -0.07591944, -0.00533499, -0.08935217, -0.0181419 ,  0.0330689 , -0.01812706, -0.07689384, -0.02715412, -0.03847084, -0.05315471, -0.02153288,  0.06898243,  0.02431128, -0.00333816,  0.00817491,  0.03911701, -0.02924617,  0.07184725, -0.00356748,  0.02246175,  0.03987982, -0.04921926,  0.02454282,  0.05875788,  0.08505439, -0.00190306, -0.03044275, -0.06383366,  0.0470311 , -0.12005549,  0.03573952, -0.04293387,  0.03283867

In [91]:
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

### 추론 수행, 정확도(accuracy) 평가

In [92]:
x_test, t_test = get_data()
network = init_network()

accuracy_cnt = 0

for i in range(len(x_test)):
    y = predict(network, x_test[i])
    p = np.argmax(y) # 확률이 가장 높은 원소의 인덱스를 얻는다.
    if p == t_test[i]:
        accuracy_cnt += 1
        
print("Accuracy:" + str(float(accuracy_cnt)/ len(x_test)))

Accuracy:0.9352


### 3.6.3 배치 처리

In [73]:
x_test, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

In [74]:
x_test.shape

(10000, 784)

In [75]:
x_test[0].shape

(784,)

In [76]:
W1.shape

(784, 50)

In [77]:
W2.shape

(50, 100)

In [78]:
W3.shape

(100, 10)

In [80]:
x_test, t_test = get_data()
network = init_network()

batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0, len(x_test), batch_size):
    x_batch = x[i:i+batch_size] # 입력 데이터 묶기
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis = 1) # 
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
    

print("Accuracy:" + str(float(accuracy_cnt)/ len(x_test)))  

Accuracy:0.9352


### np.argmax( y, axis=1) : axis
- axis 
   - 0 : 열 축을 따라 가장 높은 요소의 인덱스 찾기
   - 1 : 행 축을 따라 가장 높은 요소의 인덱스 찾기

In [83]:
x = np.array([[0.1,0.8,0.1],
              [0.3,0.1,0.6],
              [0.2,0.5,0.3],
              [0.8,0.1,0.1]])
print(x.shape)

y = np.argmax(x, axis=1) # 각 행의 열에서 가장 큰 인덱스 찾기
print(y)

(4, 3)
[1 2 1 0]


In [82]:
y2 = np.argmax(x, axis=0) # 객 열의 행에서 가장 큰 인덱스 찾기
print(y2)

[3 0 1]


#### == 연산자
- numpy 배열끼리 비교하여 True / False로 구성된 bool 배열을 만든다.
- 이 결과 배열에서 True가 몇 개인지 센다.

In [84]:
y = np.array([1,2,1,0])
t = np.array([1,2,0,0])
print(y==t)
print(np.sum(y==t))

[ True  True False  True]
3
